In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from depsurf import OUTPUT_PATH, Versions

groups = [Versions.LTS, Versions.REGULAR, Versions.REV]
# groups = [Versions.TEST]

results = []
for group in groups:
    results.append(group.diff_pairs(result_path=OUTPUT_PATH / "src" / group.name))

[        utils.py:160] INFO: NumExpr defaulting to 8 threads.
[     versions.py:148] INFO: Diffing LTS
[     versions.py:156] INFO: Comparing 4.4→4.15 to /Users/szhong/Downloads/bpf-study/output/src/LTS/4.4_4.15
[          btf.py:21 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/4.4.0-21-generic-amd64.pkl
[          btf.py:21 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/4.15.0-20-generic-amd64.pkl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.4.0-21-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.15.0-20-generic-amd64.jsonl
[     syscalls.py:54 ] WARNING: Unknown syscall at 513: ffffffff81e011a8 -> ffffffff8102cbd0
[     versions.py:156] INFO: Comparing 4.15→5.4 to /Users/szhong/Downloads/bpf-study/output/src/LTS/4.15_5.4
[          btf.py:21 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/

In [3]:
import pandas as pd
from depsurf.output import save_df

df = pd.concat(results, axis=1)
df = df.loc[(df != 0).any(axis=1)]
df = df.fillna(0).astype(int)
save_df(df, "src/summary")

[           pd.py:24 ] INFO: Saved dataframe to /Users/szhong/Downloads/bpf-study/output/src/summary.txt
[           pd.py:33 ] INFO: Saved dataframe to /Users/szhong/Downloads/bpf-study/output/src/summary.pkl


LTS                            Regular                                                                                                                                         Revision                              
                               4.4→4.15 4.15→5.4 5.4→5.15 5.15→6.8 4.4→4.8 4.8→4.10 4.10→4.13 4.13→4.15 4.15→4.18 4.18→5.0 5.0→5.3 5.3→5.4 5.4→5.8 5.8→5.11 5.11→5.13 5.13→5.15 5.15→5.19 5.19→6.2 6.2→6.5 6.5→6.8    26→51  51→75 75→100 100→125 125→175
Function   Old                    36309    42299    48018    53956   36309    38211     39148     40741     42299    44488   45277   46992   48018    51936     54302     53193     53956    55843   56978   59365    48018  48078  48138   48195   48191
           New                    42299    48018    53956    61983   38211    39148     40741     42299     44488    45277   46992   48018   51936    54302     53193     53956     55843    56978   59365   61983    48078  48138  48195   48191   48191
           Added                   8600     8929    10771    12156    2849     1574      2662      2118      3635     1980    2516    1382    6785     3518      1469      1753      3525     2622    3196    3626      102    125     97      40     189
           Removed                 2610     3210     4833     4129     947      637      1069       560      1446     1191     801     356    2867     1152      2578       990      1638     1487     809    1008       42     65     40      44     189
           Changed                 1956     1807     2187     2996     768      386       830       534       821      698     510     513     871      876       602       573       936      903    1058     801       28    174     40      19      76
           Param added             1503     1191     1506     1842     502      388       664       329       493      401     310     476     598      626       491       301       665      518     630     456       24    135     28      13      48
           Param removed           1163      895     1251     1756     388      372       463       242       281      368     294     403     518      620       258       219       596      479     641     388       11    128     16       5      25
           Param reordered          375      349      556      555     173       42       183        87       203       84     101      27     138      189       279       101       241      217     148      98        8     12     12       1       4
           Param type changed       544      460      620      862     227      120       172       162       236      182     116      29     193      275        78       216       167      287     214     331        6    126     17       2      25
           Return type changed      292      380      288      507     114       54       114        59       179      140      75      23     120       91        52        94       127      146     158     151        4     12      9       4      13
Struct     Old                     6160     7342     8422     9329    6160     6599      6752      7051      7342     7623    7807    8209    8422     8610      8954      9178      9329     9551    9767   10022     8422   8429   8438    8442    8417
           New                     7342     8422     9329    10474    6599     6752      7051      7342      7623     7807    8209    8422    8610     8954      9178      9329      9551     9767   10022   10474     8429   8438   8442    8417    8396
           Added                   1449     1364     1410     1504     536      227       371       362       393      301     467     273     480      495       323       232       384      337     318     500        8     12      5       1       8
           Removed                  267      284      503      359      97       74        72        71       112      117      65      60     292      151        99        81       162      121      63      48        1      3      1      26      29
           Changed                 1